## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [2]:
# Read data files
match_df=pd.read_csv("ODI_Matches.csv")

In [3]:
match_df.tail()

,Unnamed: 0,Scorecard,Team 1,Team 2,Margin,Ground,Match Date,Winner,Host_Country,Venue_Team1,Venue_Team2,Innings_Team1,Innings_Team2
7489,3747,ODI # 3931,New Zealand,India,Winner2ndInning,Pune,"Oct 25, 2017",India,India,Away,Home,First,Second
7490,3748,ODI # 3932,New Zealand,India,Winner1stInning,Kanpur,"Oct 29, 2017",India,India,Away,Home,Second,First
7491,3749,ODI # 3933,Namibia,Hong Kong,Winner2ndInning,Mumbai,"Aug 12, 2017",Namibia,India,Neutral,Neutral,Second,First
7492,3750,ODI # 3934,U.S.A.,Bermuda,Winner2ndInning,Mumbai,"Aug 12, 2017",U.S.A.,India,Neutral,Neutral,Second,First
7493,3749,ODI # 3935,Namibia,East Africa,Winner1stInning,Mumbai,"Aug 12, 2017",East Africa,India,Neutral,Neutral,Second,First


In [4]:
# check for null values
match_df.isnull().sum()

Unnamed: 0         0
Scorecard          0
Team 1             0
Team 2             0
Margin             0
Ground             0
Match Date         0
Winner             0
Host_Country     130
Venue_Team1        0
Venue_Team2        0
Innings_Team1      0
Innings_Team2      0
dtype: int64

In [5]:
# Coverting match_date into year,month,day
y=match_df['Match Date'].str.split(", ")
Year=[]
Month=[]
Day=[]
for i in range(7494):
    z=y[i][0]
    x=z.split(" ")
    Month.append(x[0])
    Day.append(x[1])
    Year.append(y[i][1])
    
# add column in year,month, day
match_df["Day"]=Day
match_df["Month"]=Month
match_df["Year"]=Year

# Winner Inning
Winning_Inn=[]
for i in range(7494):
    x=match_df['Margin'][i].split('r')[1]
    Winning_Inn.append(x[0])
    
match_df["WinInn"]=Winning_Inn

In [6]:
# convert object into integer
match_df["Year"]=match_df["Year"].astype('int')
match_df["WinInn"]=match_df["WinInn"].astype('int')

In [7]:
# Drop useless columns
Columns=['Unnamed: 0','Match Date','Venue_Team1','Venue_Team2','Innings_Team1','Innings_Team2','Margin']
match_df.drop(Columns,axis=1,inplace=True)

In [8]:
# Rename column
match_df.rename(columns={"Host_Country":"Host"}, inplace=True)

In [9]:
match_df.head(5)

,Scorecard,Team 1,Team 2,Ground,Winner,Host,Day,Month,Year,WinInn
0,ODI # 1,Australia,England,Melbourne,Australia,Australia,5,Jan,1971,2
1,ODI # 2,England,Australia,Manchester,England,England,24,Aug,1972,2
2,ODI # 3,England,Australia,Lord's,Australia,England,26,Aug,1972,2
3,ODI # 4,England,Australia,Birmingham,England,England,28,Aug,1972,2
4,ODI # 5,New Zealand,Pakistan,Christchurch,New Zealand,New Zealand,11,Feb,1973,1


In [10]:
match_df.drop_duplicates(subset=['Scorecard'],inplace=True)

In [11]:
# Save new file
# match_df.to_csv("new_match.csv",index=0)

# Analysis-------

In [12]:
df=pd.read_csv("new_match.csv")

In [13]:
df['Winner'].value_counts()

Australia       552
India           475
Pakistan        466
West Indies     380
Sri Lanka       371
South Africa    361
England         343
New Zealand     324
Zimbabwe        129
Bangladesh      105
Ireland          51
Afghanistan      42
Kenya            42
Scotland         31
Netherlands      28
Canada           17
U.A.E.            9
Bermuda           7
Hong Kong         6
P.N.G.            5
U.S.A.            1
Namibia           1
East Africa       1
Name: Winner, dtype: int64

In [15]:
def country_played_over_years(df,country):
    Yf=df[(df['Team 1']==country) | (df['Team 2']==country)]['Year'].value_counts().reset_index()
    Yf=Yf.rename(columns={"index":"Year","Year":"Matches Played"})
    Yf=Yf.sort_values(by=["Year"])
    return Yf

In [17]:
def country_played_with_other_countries(df,country):
    Yf=df[(df['Team 1']==country) | (df['Team 2']==country)]['Year'].value_counts().reset_index()
    Yf=Yf.rename(columns={"index":"Year","Year":"Matches Played"})
    Yf=Yf.sort_values(by=["Year"])
    return Yf


In [22]:
def country_list(df):
    countries_list=list(df['Team 1'].unique())
    countries_list.sort()
    return countries_list

In [23]:
def win_loss(df):
    new_df=df['Winner'].value_counts().reset_index()
    new_df=new_df.rename(columns={"index":"Country","Winner":"Won"})
    new_df=new_df.sort_values(by="Country")
    
    l=country_list(df)
    List=[]
    x=df
    for i in range(22):
        x=df[((df['Team 1']==l[i]) | (df['Team 2']==l[i])) & (df['Winner']!=l[i])]['Winner'].count()
        List.append(x)
    List[20]=0
    List[21]=351
    List.append(345)
    new_df['Loss']=List
    new_df["ODI_Played"]=new_df['Won']+new_df['Loss']
    return new_df

In [26]:
def opp_matches(df,country):
    Ind=df[(df['Team 1']==country) | (df['Team 2']==country)]
    x1=Ind[df['Team 1']!=country]["Team 1"].value_counts()
    x2=Ind[df['Team 2']!=country]["Team 2"].value_counts()
    z=x1.append(x2).reset_index()
    l=list(z["index"].unique())
    team=[]
    matches=[]
    for i in range(len(l)):
        m=z[z["index"]==l[i]].sum()[0]
        team.append(l[i])
        matches.append(m)
    data=pd.DataFrame()
    data["Opponet"]=team
    data["Matches"]=matches
    return data

In [113]:
def win_loss_with_countries(df,country):
    Opponent=[]
    t_win=[]
    op_win=[]
    Country_list= country_list(df)
    for i in range(len(Country_list)):
        opp=Country_list[i]
        if opp!=country:
            x1=df[(df['Team 1']==country) & (df["Team 2"]==opp)]['Winner'].value_counts().reset_index()
            x2=df[(df['Team 2']==country) & (df["Team 1"]==opp)]['Winner'].value_counts().reset_index()
            z=x1.append(x2)
            team=z[z['index']==country].sum()["Winner"]
            opp_win=z[z['index']==opp].sum()["Winner"]
            Opponent.append(opp)
            op_win.append(opp_win)
            t_win.append(team)
    data=pd.DataFrame()
    data['Opponent']=Opponent
    data["Opp_win"]=op_win
    data["team_win"]=t_win
    
    data["Opp_win"]=data["Opp_win"].astype('int')
    data["team_win"]=data["team_win"].astype('int')

    data["Total_matches"]=data["Opp_win"]+data["team_win"]
    return data